In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2


from utils import make_video, process_cams

/home/cat/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datetime import datetime, timezone, timedelta

#############################################
#############################################
#############################################
n_cams = 18
root_dir = "/home/cat/Downloads/data_stitching/cams/"
date = "2025_07_31"                               # date of the video
hour_start = "00"                                     # start at midnight
n_mins = 60
shrink_factor = 10   # this shrinks the video along x and y axis by this factor; for now we use subsampling
frame_subsample = 4
cage_id = 1
cam_ids = np.arange(1, n_cams + 1)

#
skip_regeneration = False  # if True, delete old bin files before making new ones
parallel_flag = False
delete_bins = False
build_video_only = False

# outer loop gooin over every minue of time 
for minute in range(n_mins):

    # save the combined frame as a video file
    fname_combined = os.path.join(root_dir, "minute_" +str(minute)+'.avi')

    #
    if os.path.exists(fname_combined):
        print ("Video exists for Minute ", str(minute), "...skipping...")
        print ('')
        print ('')
        continue

    #
    if build_video_only==False:
        process_cams(cam_ids,
                    root_dir,
                    cage_id,
                    date,
                    hour_start,
                    minute,
                    parallel_flag=parallel_flag,
                    shrink_factor=shrink_factor,
                    skip_regeneration=False)

    #################################################
    ############### Make video ######################
    #################################################
    # process #2 - here we make the mosaic 1 minute video based on the available files
    # we loop over all possible files 
    overwrite_existing = False
    make_video(root_dir,
               date,
               minute,
               n_cams,
               fname_combined,
               shrink_factor=shrink_factor,
               frame_subsample=frame_subsample,
               overwrite_existing=overwrite_existing,
               delete_bins = delete_bins)
    
    print ("***************************")
    print ('')

    

Video exists for Minute  0 ...skipping...


Video exists for Minute  1 ...skipping...


Video exists for Minute  2 ...skipping...


Video exists for Minute  3 ...skipping...


Video exists for Minute  4 ...skipping...


Video exists for Minute  5 ...skipping...


Video exists for Minute  6 ...skipping...


Video exists for Minute  7 ...skipping...


Video exists for Minute  8 ...skipping...


Video exists for Minute  9 ...skipping...


Video exists for Minute  10 ...skipping...


Video exists for Minute  11 ...skipping...


Video exists for Minute  12 ...skipping...


Video exists for Minute  13 ...skipping...


Video exists for Minute  14 ...skipping...


Video exists for Minute  15 ...skipping...


Video exists for Minute  16 ...skipping...


Video exists for Minute  17 ...skipping...


... processing camera:  1  for minute:  18
... processing camera:  2  for minute:  18
... processing camera:  3  for minute:  18
... processing camera:  4  for minute:  18
... processing camera:  5  f

  0%|          | 0/1500 [00:00<?, ?it/s]

100%|██████████| 1500/1500 [00:13<00:00, 107.92it/s]

finished writing combined video file:  /home/cat/Downloads/data_stitching/cams/minute_18.avi
***************************

Video exists for Minute  19 ...skipping...


Video exists for Minute  20 ...skipping...


Video exists for Minute  21 ...skipping...


Video exists for Minute  22 ...skipping...


Video exists for Minute  23 ...skipping...


Video exists for Minute  24 ...skipping...


Video exists for Minute  25 ...skipping...


Video exists for Minute  26 ...skipping...


Video exists for Minute  27 ...skipping...


Video exists for Minute  28 ...skipping...


Video exists for Minute  29 ...skipping...


Video exists for Minute  30 ...skipping...


Video exists for Minute  31 ...skipping...


Video exists for Minute  32 ...skipping...


Video exists for Minute  33 ...skipping...


Video exists for Minute  34 ...skipping...


Video exists for Minute  35 ...skipping...


Video exists for Minute  36 ...skipping...


Video exists for Minute  37 ...skipping...


Video exists for Minute

In [15]:
import numpy as np

rows = np.array([
    [16, 13, 10, 7, 4, 1],
    [17, 14, 11, 8, 5, 2],
    [18, 15, 12, 9, 6, 3]
])

cam = 1

result = np.where(rows == cam)
print("result:", result)

if result[0].size > 0:
    row_index = result[0][0]
    col_index = result[1][0]
    print("row:", row_index, "col:", col_index)
else:
    print("Not found")



result: (array([0]), array([5]))
row: 0 col: 5


In [ ]:
# 
import yaml

n_cams = 18
root_dir = "/home/cat/Downloads/data_stitching/cams/"
date = "2025_07_31"                               # date of the video
hour_start = "07"                                     # start at midnight
n_mins = 60
shrink_factor = 10   # this shrinks the video along x and y axis by this factor; for now we use subsampling
cage_id = 1
cam_ids = np.arange(1, n_cams + 1)

#
skip_regeneration = True  # if True, delete old bin files before making new ones
parallel_flag = False
delete_bins = False
build_video_only = True


fname = "camera_layout.yaml"
cam_coordinates = yaml.safe_load(open(fname, 'r'))
print ("cam_coordinate: ", cam_coordinates)

rows = [
        [16,13,10,7,4,1],
        [17,14,11,8,5,2],
        [18,15,12,9,6,3]
]

rows = np.array(rows)

# load translation table
fname = os.path.join(root_dir, "translation_table.yaml")
translations = yaml.safe_load(open(fname, 'r'))

################################################################
# ok so need to accumulate the x - row-wise translations
trans_cum_x_array = []
for cam in range(1,n_cams+1,1):
    trans = translations['cam'+str(cam)]
   # print (trans)

    # find the row of the camera
    for ctr, row in enumerate(rows):
        idx = np.where(row==cam)[0]
        #print ("row: ", row, "cam: ", cam)
        if idx.shape[0]!=0:
            #print ("found row: ", ctr, ", and column: ", idx)
            # now need to sum all the x translations from all the cameras to the left
            temp_x = 0
            for i in range(idx[0]+1):
                temp_cam = translations['cam'+str(row[i])]
                #print ('cam'+str(row[i]) + ", adding shift: ", temp_cam)
                temp_x+= temp_cam[0]
            print ("total shift for cam: ", cam, "is ", temp_x)
            #print (cam_coordinates['cam'+str(row[i])])
            temp_x+=cam_coordinates['cam'+str(row[i])][0]
            print ("final location cam: ", cam, "is ", temp_x)

    # 
    trans_cum_x_array.append(temp_x)


    


cam_coordinate:  {'cam1': [6400, 0], 'cam2': [6400, 720], 'cam3': [6400, 1440], 'cam4': [5120, 0], 'cam5': [5120, 720], 'cam6': [5120, 1440], 'cam7': [3840, 0], 'cam8': [3840, 720], 'cam9': [3840, 1440], 'cam10': [2560, 0], 'cam11': [2560, 720], 'cam12': [2560, 1440], 'cam13': [1280, 0], 'cam14': [1280, 720], 'cam15': [1280, 1440], 'cam16': [0, 0], 'cam17': [0, 720], 'cam18': [0, 1440]}
total shift for cam:  1 is  -330
final location cam:  1 is  6070
total shift for cam:  2 is  -330
final location cam:  2 is  6070
total shift for cam:  3 is  -330
final location cam:  3 is  6070
total shift for cam:  4 is  -150
final location cam:  4 is  4970
total shift for cam:  5 is  -150
final location cam:  5 is  4970
total shift for cam:  6 is  -150
final location cam:  6 is  4970
total shift for cam:  7 is  0
final location cam:  7 is  3840
total shift for cam:  8 is  0
final location cam:  8 is  3840
total shift for cam:  9 is  0
final location cam:  9 is  3840
total shift for cam:  10 is  0
fin

In [ ]:
fname = '/home/cat/Downloads/data_stitching/1/minute_1.bin'

# let's load this using mmap as a binary file
data = np.memmap(fname, dtype=np.uint8, mode='r', shape=(6000, 720, 1280, 3))

# let's check the shape of the data
print("Data shape:", data.shape)

# ok now let's grab a frame and display it
idx = 2000
frame = data[idx]

# let's display the frame using opencv
cv2.imshow("Frame", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


Data shape: (6000, 720, 1280, 3)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/cat/miniconda3/lib/python3.11/site-packages/cv2/qt/plugins"


In [ ]:
# Ignore - past here

In [110]:
fname0 = '/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_00_41_557_metadata.npz'
fname2 = '/home/cat/Downloads/data_stitching/cams/2/1_2_2025_07_31_07_01_44_905_metadata.npz'

frame_times1 = np.load(fname1)['frame_times']
frame_times2 = np.load(fname2)['frame_times']
print ((frame_times1 - 1753941704894859)//1000+49880, "msecs")
print ((frame_times2 - 1753941704894859)//1000+49880-60000, "msecs")


[ 49880  49888  49896 ... 109864 109872 109881] msecs
[-10120 -10112 -10104 ...  49864  49872  49881] msecs


In [109]:
fname_bin1 = '/home/cat/Downloads/data_stitching/cams/2/minute_1.bin'
data_bin1 = np.fromfile(fname_bin1).reshape(32,18,3)
# print size of file
print (data_bin1.size/32/18)

print (data_bin1.shape)

ValueError: cannot reshape array of size 897480 into shape (32,18,3)

In [ ]:


#########################################
# LOAD video time stamps
data = np.load(fnames_cam1[0], allow_pickle=True)
print (data.files)
frame_times_ms = data['frame_times']//1000
print (frame_times_ms)
recording_start_time = data['recording_start_time']
print ("recording start time: ", recording_start_time)
encoder_start = data['encoder_start']
print ("encoder start: ", encoder_start)

###############################################
# convert july 24, 2025  exacdtly midngith to milisecond in epoch systm eimte clock but make sure its' UTC+1 london time
epoch_start = np.datetime64('2025-07-24T00:00:00', 'ms') - np.timedelta64(1, 'h')  # UTC+1
epoch_start_ms = epoch_start.astype('datetime64[ms]').astype(int)

# so this is the video time stamps relative to the epoch start
delta_times = frame_times_ms - epoch_start_ms
print("time srelative to midnight (in ms): ", delta_times)

# now convert into bucket discrete time
delta_times_bucket = delta_times // 10 * 10 # convert to seconds
print("time relative to midnight in 10ms buckets: ", delta_times_bucket)

# and convert into a discrete bin of 10ms from midnight
delta_times_bucket_discrete = delta_times_bucket // 10
print("time relative to midnight in 10ms discrete buckets: ", delta_times_bucket_discrete)
print ('')
print (' So we either ruse this bueckt version which tells you which 10ms bucket the frame is in')
print (' or the disrete version which tells you exactly what frame of the video to put the uncompressed video data into')

In [51]:
#####################################
# make some fake video data but don't generate the full array as it will take 100GB of ram
data_cam1_vid1 = np.zeros((frame_times_ms.shape[0]))
print ("video data (ordinarlily this would be an .mp4 that's decompressed on the fly: ", data_cam1_vid1.shape)

# and we crate a 1 min video bucket to hold the videos
stitched_video = np.zeros((18, 60000))
print ("stitched video (18 rpis, 60000 times should be about 10minutes (100fps * 60 * 10)): ", stitched_video.shape)
print (" (the full video is much larger as it has the frames 1024 , 768 and 3 channels for RGB)")

# so now we can loop over the relative to midnight time dicscrete buickets
for i in range(delta_times_bucket_discrete.shape[0]):
    bucket = delta_times_bucket_discrete[i]
    if i%1000==0:
        print("processing frame ", i, " in bucket ", bucket)
    
    # so now we can put the video data into the stitched video
    # this is just a fake example, in reality you would decompress the video and put it into the stitched video
    stitched_video[0, bucket] = data_cam1_vid1[i]  # assuming we are putting cam1 data into the first row


video data (ordinarlily this would be an .mp4 that's decompressed on the fly:  (7205,)
stitched video (18 rpis, 60000 times should be about 10minutes (100fps * 60 * 10)):  (18, 60000)
 (the full video is much larger as it has the frames 1024 , 768 and 3 channels for RGB)
processing frame  0  in bucket  7589
processing frame  1000  in bucket  8422
processing frame  2000  in bucket  9255
processing frame  3000  in bucket  10088
processing frame  4000  in bucket  10921
processing frame  5000  in bucket  11754
processing frame  6000  in bucket  12587
processing frame  7000  in bucket  13420


In [19]:
print("Files:", data.files)
print("First 5 frame_times_ms:", frame_times_ms[:5])
print("Recording start time:", recording_start_time)
print("Encoder start:", encoder_start)
print("Midnight reference:", epoch_start_ms)
print("First timestamp:", frame_times_ms[0])
print("Delta (first):", delta_times[0])


Files: ['frame_times', 'frames_encoded', 'recording_start', 'recording_end', 'encoder_start', 'frame_rate', 'recording_duration', 'recording_mode', 'recording_type', 'recording_duration_continuous', 'recording_duration_triggered', 'recording_start_time']
First 5 frame_times_ms: [1753311675898 1753311675906 1753311675914 1753311675923 1753311675931]
Recording start time: 2025-07-24_00-01-15
Encoder start: 1753311675.907574
Midnight reference: 1753315200000
First timestamp: 1753311675898
Delta (first): -3524102


In [20]:
print("Recording start time (converted):", 
      np.datetime64(int(recording_start_time), 'ms'))


ValueError: invalid literal for int() with base 10: '2025-07-24_00-01-15'